In [1]:
import time

import syft as sy
from syft.client.domain_client import DomainClient
from syft.client.syncing import compare_clients
from syft.service.code.user_code import UserCode
from syft.service.job.job_stash import Job, JobStatus
from syft.service.request.request import Request
from syft.service.sync.diff_state import ObjectDiffBatch


def is_request_to_sync(batch: ObjectDiffBatch) -> bool:
    # True if this is a new low-side request
    # TODO add condition for sql requests/usercodes
    low_request = batch.root.low_obj
    return isinstance(low_request, Request) and batch.status == "NEW"


def is_job_to_sync(batch: ObjectDiffBatch):
    # True if this is a new high-side job that is either COMPLETED or ERRORED
    if batch.status != "NEW":
        return False
    if not isinstance(batch.root.high_obj, Job):
        return False
    job = batch.root.high_obj
    return job.status in (JobStatus.ERRORED, JobStatus.COMPLETED)


def sync_new_requests(
    client_low: DomainClient,
    client_high: DomainClient,
) -> dict[sy.UID, sy.SyftSuccess | sy.SyftError] | sy.SyftError:
    sync_request_results = {}
    diff = compare_clients(
        from_client=client_low, to_client=client_high, include_types=["request"]
    )
    if isinstance(diff, sy.SyftError):
        print(diff)
        return sync_request_results
    for batch in diff.batches:
        if is_request_to_sync(batch):
            request_id = batch.root.low_obj.id
            w = batch.resolve()
            result = w.click_sync()
            sync_request_results[request_id] = result
    return sync_request_results


def execute_requests(
    client_high: DomainClient, request_ids: list[sy.UID]
) -> dict[sy.UID, Job]:
    jobs_by_request_id = {}
    for request_id in request_ids:
        request = client_high.requests.get_by_uid(request_id)
        if not isinstance(request, Request):
            continue

        code = request.code
        if not isinstance(code, UserCode):
            continue

        func_name = request.code.service_func_name
        api_func = getattr(client_high.code, func_name, None)
        if api_func is None:
            continue

        job = api_func(blocking=False)
        jobs_by_request_id[request_id] = job
        # sleep to prevent SQLite connection pool issues
        time.sleep(1)

    return jobs_by_request_id


def sync_and_execute_new_requests(
    client_low: DomainClient, client_high: DomainClient
) -> None:
    sync_results = sync_new_requests(client_low, client_high)
    if isinstance(sync_results, sy.SyftError):
        print(sync_results)
        return

    request_ids = [
        uid for uid, res in sync_results.items() if isinstance(res, sy.SyftSuccess)
    ]
    print(f"Synced {len(request_ids)} new requests")

    jobs_by_request = execute_requests(client_high, request_ids)
    print(f"Started {len(jobs_by_request)} new jobs")


def sync_finished_jobs(
    client_low: DomainClient,
    client_high: DomainClient,
) -> dict[sy.UID, sy.SyftError | sy.SyftSuccess] | sy.SyftError:
    sync_job_results = {}
    diff = compare_clients(
        from_client=client_high, to_client=client_low, include_types=["job"]
    )
    if isinstance(diff, sy.SyftError):
        print(diff)
        return diff

    for batch in diff.batches:
        if is_job_to_sync(batch):
            batch_id = batch.root.high_obj.id
            w = batch.resolve()
            share_result = w.click_share_all_private_data()
            if isinstance(share_result, sy.SyftError):
                sync_job_results[batch_id] = share_result
                continue
            sync_result = w.click_sync()
            sync_job_results[batch_id] = sync_result

    print(f"Sharing {len(sync_job_results)} new results")
    return sync_job_results


def auto_sync(client_low: DomainClient, client_high: DomainClient) -> None:
    print("Starting auto sync")
    sync_and_execute_new_requests(client_low, client_high)
    sync_finished_jobs(client_low, client_high)
    print("Finished auto sync")


def auto_sync_loop(
    client_low: DomainClient, client_high: DomainClient, sleep_seconds: int = 60
) -> None:
    while True:
        auto_sync(client_low, client_high)
        time.sleep(sleep_seconds)

In [2]:
low_side = sy.orchestra.launch(
    name="low-side",
    node_side_type="low",
    local_db=True,
    reset=True,
    dev_mode=True,
)

high_side = sy.orchestra.launch(
    name="high-side",
    node_side_type="high",
    local_db=True,
    reset=True,
    n_consumers=4,
    create_producer=True,
    dev_mode=True,
)

client_high = high_side.login(email="info@openmined.org", password="changethis")
client_low = low_side.login(email="info@openmined.org", password="changethis")
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = low_side.login(email="newuser@openmined.org", password="pw")

Staging Protocol Changes...
Document Store's SQLite DB path: /var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/syft/8fb2b26c3b5d4db2a0cb775fe2a3d825/db/8fb2b26c3b5d4db2a0cb775fe2a3d825.sqlite
Action Store's SQLite DB path: /var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/syft/8fb2b26c3b5d4db2a0cb775fe2a3d825/db/8fb2b26c3b5d4db2a0cb775fe2a3d825.sqlite
Creating default worker image with tag='local-dev'
Setting up worker poolname=default-pool workers=0 image_uid=ecdf0b0b455f423a9c5c84b1f51be2b2 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
Document Store's SQLite DB path: /var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/syft/083dfc0ecd744d17ad21a36a6477565e/db/083dfc0ecd744d17ad21a36a6477565e.sqlite
Action Store's SQLite DB path: /var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/syft/083dfc0ecd744d17ad21a36a6477565e/db/083dfc0ecd744d17ad21a36a6477565e.sqlite
Creating default worker image with tag='local-dev'
Setting up worker poo

SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <low-side: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <low-side: Low side Domain> as <newuser@openmined.org>


In [3]:
def make_request(client):
    existing_requests = client.requests.get_all()

    @sy.syft_function_single_use()
    def func():
        return 10

    func.func_name = f"query_{len(existing_requests)}"
    func.code = func.code.replace("def func(", f"def {func.func_name}(")

    res = client.code.request_code_execution(func)
    return res


for _ in range(5):
    make_request(client_low_ds)

SyftSuccess: Syft function 'func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [4]:
auto_sync(client_low, client_high)

Starting auto sync


SyftWarning: User code is hidden by default, as they are also part of the Request. If you want to include them, set hide_usercode=False.

Decision: Syncing 2 objects
Decision: Syncing 2 objects
Decision: Syncing 2 objects
Decision: Syncing 2 objects
Decision: Syncing 2 objects
Synced 5 new requests
START Setting Job 0588393eb0bc4e74a788cc48780a60e2, thread 8023429120
END Setting Job 0588393eb0bc4e74a788cc48780a60e2, thread 8023429120
START Updating Job 0588393eb0bc4e74a788cc48780a60e2, thread 11293306880
END Updating Job 0588393eb0bc4e74a788cc48780a60e2, thread 11293306880, res: Ok(syft.service.job.job_stash.Job), obj: syft.service.job.job_stash.Job
Job 0588393eb0bc4e74a788cc48780a60e2 found: Ok(syft.service.job.job_stash.Job)
START Updating Job 0588393eb0bc4e74a788cc48780a60e2, thread 11633979392
END Updating Job 0588393eb0bc4e74a788cc48780a60e2, thread 11633979392, res: Ok(syft.service.job.job_stash.Job), obj: syft.service.job.job_stash.Job
Job 0588393eb0bc4e74a788cc48780a60e2 found: Ok(syft.service.job.job_stash.Job)
START Setting Job c586ffdd174d441eb7c1ca23633629b1, thread 8023429120
END Setting Job c586ffdd174d441e

SyftWarning: User code is hidden by default, as they are also part of the Request. If you want to include them, set hide_usercode=False.

START Updating Job 0537c2e924334bedbd2b641625905cc4, thread 11639091200
END Updating Job 0537c2e924334bedbd2b641625905cc4, thread 11639091200, res: Ok(syft.service.job.job_stash.Job), obj: syft.service.job.job_stash.Job
Job 0537c2e924334bedbd2b641625905cc4 found: Ok(syft.service.job.job_stash.Job)
Decision: Syncing 5 objects
START Setting Job f45877b68917412bb85b806e179871ed, thread 8023429120
END Setting Job f45877b68917412bb85b806e179871ed, thread 8023429120
Decision: Syncing 5 objects
START Setting Job 0588393eb0bc4e74a788cc48780a60e2, thread 8023429120
END Setting Job 0588393eb0bc4e74a788cc48780a60e2, thread 8023429120
Decision: Syncing 5 objects
START Setting Job c586ffdd174d441eb7c1ca23633629b1, thread 8023429120
END Setting Job c586ffdd174d441eb7c1ca23633629b1, thread 8023429120
Decision: Syncing 5 objects
START Setting Job cc2aaa7f2c7542a69e3fac248d5acd8d, thread 8023429120
END Setting Job cc2aaa7f2c7542a69e3fac248d5acd8d, thread 8023429120
Sharing 4 new results
Finished auto s

In [10]:
for _ in range(10):
    j = client_high.code.query_0(blocking=False)
    print(j)

START Setting Job 504dfa7a0309408dad11188df0a267e9, thread 8023429120
END Setting Job 504dfa7a0309408dad11188df0a267e9, thread 8023429120
syft.service.job.job_stash.Job
START Setting Job c9d754572ff14cc285c01b4b4e8bb86e, thread 8023429120
END Setting Job c9d754572ff14cc285c01b4b4e8bb86e, thread 8023429120
syft.service.job.job_stash.Job
START Setting Job 449c70aaecc4479f92835cf633985815, thread 8023429120
END Setting Job 449c70aaecc4479f92835cf633985815, thread 8023429120
syft.service.job.job_stash.Job
START Setting Job da0238a7f7794ef79bc49b987cf2f22d, thread 8023429120
END Setting Job da0238a7f7794ef79bc49b987cf2f22d, thread 8023429120
syft.service.job.job_stash.Job
START Setting Job 099e40c538e44232969b439589226e10, thread 8023429120
END Setting Job 099e40c538e44232969b439589226e10, thread 8023429120
syft.service.job.job_stash.Job
START Setting Job 011a06deaf05409983cc924cfc13f8fe, thread 8023429120
END Setting Job 011a06deaf05409983cc924cfc13f8fe, thread 8023429120
syft.service.job.

In [30]:
high_side.python_node.job_stash.query_one(sy.UID("72a80389d09043e087aa5be880df38e8"), credentials=client_high.verify_key)

TypeError: BaseStash.query_one() got multiple values for argument 'credentials'

In [36]:
from syft.store.document_store import QueryKeys, UIDPartitionKey

job_stash = high_side.python_node.job_stash
credentials = client_high.verify_key
uid = sy.UID("72a80389d09043e087aa5be880df38e8")
qks = QueryKeys(qks=[UIDPartitionKey.with_obj(uid)])
r = job_stash.query_one(credentials=credentials, qks=qks)

r

Ok(None)

In [59]:
job_stash.partition.unique_keys["id"]

{<UID: c5f05da93bc84be59db27d8bc4a0daee>: <UID: c5f05da93bc84be59db27d8bc4a0daee>,
 <UID: 9776b0dc3f824c6d8b679ec4602d3d5e>: <UID: 9776b0dc3f824c6d8b679ec4602d3d5e>,
 <UID: 9471af438652410180a83de78d19ba51>: <UID: 9471af438652410180a83de78d19ba51>,
 <UID: 818cb76dd1e543758043b8c503098d46>: <UID: 818cb76dd1e543758043b8c503098d46>,
 <UID: aaacb85816384e389252f65685b3882f>: <UID: aaacb85816384e389252f65685b3882f>,
 <UID: 88334511ed934ddc87aa0e73c7c96e7e>: <UID: 88334511ed934ddc87aa0e73c7c96e7e>,
 <UID: bb3a085a5ed2441c9b55aa48f94e85f2>: <UID: bb3a085a5ed2441c9b55aa48f94e85f2>,
 <UID: e47c51591b11441a9ab94a013578f5c0>: <UID: e47c51591b11441a9ab94a013578f5c0>,
 <UID: 1a5379a299324a37adca6f2143b1ca1e>: <UID: 1a5379a299324a37adca6f2143b1ca1e>,
 <UID: b981079d7912490d99d28b5c282d3cfa>: <UID: b981079d7912490d99d28b5c282d3cfa>,
 <UID: 972d3f98c0a941b1a26c8f3cf9bf0e27>: <UID: 972d3f98c0a941b1a26c8f3cf9bf0e27>,
 <UID: 01352af2cf58431797f79e02d4301c40>: <UID: 01352af2cf58431797f79e02d4301c40>,
 <UI

In [47]:
%debug

> /Users/eelco/.pyenv/versions/3.10.13/lib/python3.10/uuid.py(177)__init__()
    175             hex = hex.strip('{}').replace('-', '')
    176             if len(hex) != 32:
--> 177                 raise ValueError('badly formed hexadecimal UUID string')
    178             int = int_(hex, 16)
    179         if bytes_le is not None:



ipdb>  value


*** NameError: name 'value' is not defined


ipdb>  u


> /Users/eelco/dev/PySyft/packages/syft/src/syft/types/uid.py(71)__init__()
     69         # if value is not set - create a novel and unique ID.
     70         if isinstance(value, str):
---> 71             value = uuid.UUID(value, version=4)
     72         elif isinstance(value, bytes):
     73             value = uuid.UUID(bytes=value, version=4)



ipdb>  value


'id'


ipdb>  q


In [43]:
job_stash.partition.matches_unique_cks(qks.all[0].partition_key)

True

In [31]:
high_side.python_node.job_stash.query_one?

Signature:
high_side.python_node.job_stash.query_one(
    credentials: 'SyftVerifyKey',
    qks: 'QueryKey | QueryKeys',
    order_by: 'PartitionKey | None' = None,
) -> 'Result[BaseStash.object_type | None, str]'
Docstring: <no docstring>
File:      ~/dev/PySyft/packages/syft/src/syft/store/document_store.py
Type:      method

In [18]:
p.data[sy.UID("72a80389d09043e087aa5be880df38e8")]

syft.service.job.job_stash.Job

In [24]:
p._get(sy.UID("72a80389d09043e087aa5be880df38e8"), credentials=client_high.verify_key).ok()

syft.service.job.job_stash.Job

AttributeError: 'SQLiteStorePartition' object has no attribute 'query_one'

In [25]:
client_high.services.job.get(sy.UID("72a80389d09043e087aa5be880df38e8"))

In [17]:
client_high.services.job.get(sy.UID("830b9e1fd2bf4bf9b526264e3468f97c"))

In [7]:
import sqlite3

conn = sqlite3.connect(database="x")

In [9]:
cursor = conn.cursor()

In [11]:
conn.close()

In [12]:
cursor.execute("x")

ProgrammingError: Cannot operate on a closed database.

In [6]:
a = {}

del a[1]

KeyError: 1

In [11]:
client_high.jobs[0]

syft.service.job.job_stash.Job